# BASIN moveout plot notebooks
This notebook makes moveout plots of the stacked cross correlations between broadbands and nodal stations. It is highly inspired by Julian Schmitt's work, modified by Marine Denolle (mdenolle@uw.edu)

In [11]:
# Packages - set GR as plotting backend 
#include("/Users/julianschmitt/Documents/Schoolwork/Seismology/SeisCore.jl/src/SeisCore.jl")
using Pkg,  HDF5, SeisIO, SeisNoise, Plots, Dates, CSV, DataFrames, Distributed, JLD2, Statistics, PyCall, Glob, StructArrays, ColorSchemes, Plots.PlotUtils, AbstractFFTs
# loadcolorscheme(:cm_maxamp,ColorSchemes.gist_heat.colors[end-50:-1:1], "maxamp color", "for waveform plot");

In [13]:
include("../../src/process.jl")
include("/Users/marinedenolle/BASIN/C4-Project.jl/src/process.jl")
include("../src/utils.jl")
ENV["GR"] = ""
Pkg.build("GR")

LoadError: The following package names could not be resolved:
 * GR (not found in project or manifest)


## function to plot per distance range

In [24]:
function plot_by_latitude(corrs::Array{CorrData,1}, freqs::Array{Array{Float64,1},1}, 
                            attr::Array{String,1}=["","","",""], src_geo::Any=nothing, fs::Float64=20.)
    println("crap!")
#     loadcolorscheme(:cm_maxamp,ColorSchemes.hot.colors[end-50:-1:1], "maxamp color", "for waveform plot");
#     loadcolorscheme(:cm_maxamp,ColorSchemes.jet1.colors[end-50:-1:1], "maxamp color", "for waveform plot");
    loadcolorscheme(:cm_maxamp,ColorSchemes.jet1.colors[end:-1:1], "maxamp color", "for waveform plot");
    source = attr[1]
    println(source)
    println("Sorting by distance")
    plot_sorttype == "Latitude"
    println(corrs[1].loc.lat)
    
    sort_method = sort([corr.loc.lat for corr in corrs])
#         sort_method = sort([corr.latitude for corr in corrs])#sort([get_dist(source_geo, corr.loc) for corr in corrs])
    p = sortperm([corr.loc.lat for corr in corrs])#[get_dist(source_geo, corr.loc) for corr in corrs])
        #println("Not currently a supported stacktype")


    lmin, lmax = minimum(sort_method), maximum(sort_method)
    new_corrs, n_corrs = corrs[p], length(corrs)
    y_labels = (round.(collect(range(0, 10*n_corrs, length=10)),digits=3),round.(collect(range(lmin, lmax, length=10)),digits=3))
    for freq_pair in freqs
        NS_plot = plot(xlims = (-100,100), ylims = (-10,n_corrs*10+15), 
                    yticks=y_labels,xlabel = "Time (s)", ylabel = "Station Nodes by $plot_sorttype (lat)",  
                    xtickfontsize=5,ytickfontsize=5,fontsize=5, xguidefontsize = 10, yguidefontsize = 10,
                    legendfontsize = 10, title="$(attr[3]) $(attr[2]) from $(attr[1]): $(freq_pair[1]) to $(freq_pair[2])",
                    titlefontsize= 10)
        # bandpass correlations 
        corrs_processed = []
        max_amp = []
        ys, maxloc = [], []
        for (ind, corr) in enumerate(new_corrs)
            try
                # bandpass, scale, shift, and add to plot
                dist = corr.dist #ind*5#get_dist(source_geo, corr.loc)
                taper!(corr)
                processed = bandpass(fftderivative(corr).corr, freq_pair[1], freq_pair[2], 20.)
                nwin2 = convert(Int64, round(length(processed)/2))
                t2=convert(Int64, floor(nwin2-(dist/5)*corr.fs))
                t1=convert(Int64, ceil(nwin2-(dist/1.5)*corr.fs))
                trange = collect(t1:1:t2) # anticausal window of ballistic waves
                imax = findmax(processed[trange])[2]+t1
                push!(maxloc, imax)
                push!(ys, ind*10)
                push!(max_amp, findmax(processed[trange]))
                push!(corrs_processed, processed./findmax(processed)[1].*5 )
            catch e
                println(e)
            end
        end
        # get the max amp
        mmax = findmax(max_amp)[1][1]
        T = collect(-300.:1/fs:300.)
        crap=[]
        for i in collect(1:length(corrs_processed))
            push!(crap,corrs_processed[i]./max_amp[i][1].*mmax .+ 10*i)

        end
        Plots.plot!(NS_plot, T, crap, color=:cm_maxamp,  
                line_z=getindex.(max_amp,1)', fmt = :png, linewidth = 0.5, reuse = false, legend = false)  
#         plot!(NS_plot, T, crap, color=:cm_maxamp, colorbar_title="Normalized Maximum Amplitude", 
#                 line_z=getindex.(max_amp,1)', fmt = :png, linewidth = 0.5, reuse = false, legend = false)     
        Plots.plot!(size=(350,400),dpi=500)
        
        # add dots where max_amp is chosen 
        dot_locations = [amp[2] for amp in max_amp]
        y = 10 .* collect(0:length(max_amp))
        mloc = [T[ind] for ind in maxloc]
        scatter!(mloc, ys, color="black", markersize=1.2)
        filepath = joinpath("/Users/marinedenolle/BASIN/nodestack_plots/$(attr[1])/")
        DIR = dirname(filepath)
        if !isdir(DIR)
            mkpath(DIR)
        end
        print(filepath)
        png(NS_plot,filepath*"$(new_corrs[1].name[1:5])_waveforms_$(attr[2])_$(freq_pair[1])to$(freq_pair[2])_$(attr[4]).png")
    end
end

plot_by_latitude (generic function with 4 methods)

In [15]:
?fftderivative

search: fftderivative



No documentation found.

`fftderivative` is a `Function`.

```
# 1 method for generic function "fftderivative":
[1] fftderivative(A::CorrData) in Main at /Users/marinedenolle/BASIN/C4-Project.jl/src/process.jl:5
```


In [16]:
# metaparams for plots
n_derivatives = 2
n_deriv_names = ["Raw","PGD","PDV"]
#plot_sorttype = "Latitude" # other options include "Longitude" and (soon) "Distance" (in progress...) 
years = [2017]#, 2018, 2019]
components = ["ZZ","EE","EN","EZ","NE", "NN", "NZ","ZE","ZN", "ZZ"]
source_stations = ["SVD"]#["CHN","CJM","DEV","IPT","LPC","SNO","SVD","TA2"]
rootdir = "/Users/marinedenolle/BASIN" # chose root directory for plots
lw, scale, frequency_plots = 0.5, 3, [[0.1,0.35]] #  [[0.1,0.2],[0.15, 0.25],[0.2,0.3],[0.1,0.35]]
locations = DataFrame(CSV.File("/Users/marinedenolle/BASIN/C4-Project.jl/docs/modified_nodal.csv"));

In [17]:
files = glob("nodestack/*",rootdir);
println(files)
filter!(x -> occursin("SVD", x), files)

["/Users/marinedenolle/BASIN/nodestack/2017", "/Users/marinedenolle/BASIN/nodestack/2018", "/Users/marinedenolle/BASIN/nodestack/2019", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_allday.h5", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_ref.h5", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp.h5", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp_night.h5", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp_night_shortwindows.h5", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_strict.h5"]


6-element Vector{String}:
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_allday.h5"
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_ref.h5"
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp.h5"
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp_night.h5"
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp_night_shortwindows.h5"
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_strict.h5"

In [28]:
# print(files)
f=files[1]
println(typeof(f))
year = split(split(split(f,"/")[end],"_")[end-1],".")[1]
println(year)
println(f)
processingtype = String(split(split(split(f,"/")[end],"_")[end],".")[1])
println(processingtype)

String
2017
/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_allday.h5
allday


## Stackplot Script for BASIN Data

In [29]:
plot_sorttype = "Latitude"
println(f)
# year = split(split(split(f,"/")[end],"_")[end-1],".")[1]
# year = split(split(split(f,"/")[end],"_")[end-1],".")[1]
println(year)
# 
try  
    file = h5open(f,"r")
    for comp in components
        for (ind, sktype) in enumerate(["linear"])#,"pws","robust"])
            for filt in ["NO.B4","NO.G1","NO.G2"]#,"NO.B2","NO.B3", "NO.B4","NO.B5","NO.B6","NO.G1","NO.G2","NO.G3","NO.G4"]
                source, stacktype, component, filter = convert(String, split(f, ".")[end-1][1:3]), sktype, comp, filt
                fcorrs = get_corrs(file, stacktype, component, filter)
                if length(fcorrs) > 1
#                         println("Plotting $filt $comp $sktype for $source in $year")
                    attributes = [source,component,stacktype,String(processingtype)]
                    plot_by_latitude(fcorrs, frequency_plots,attributes)
#                         println("finised plotting")
                end
            end
        end
    end
catch e
    println(e)
end

/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_allday.h5
2017
cm_maxamp overwritten
SVD
Sorting by distance
/Users/marinedenolle/BASIN/nodestack_plots/SVD/cm_maxamp overwritten
SVD
Sorting by distance
/Users/marinedenolle/BASIN/nodestack_plots/SVD/cm_maxamp overwritten
SVD
Sorting by distance
/Users/marinedenolle/BASIN/nodestack_plots/SVD/cm_maxamp overwritten
SVD
Sorting by distance
/Users/marinedenolle/BASIN/nodestack_plots/SVD/cm_maxamp overwritten
SVD
Sorting by distance
/Users/marinedenolle/BASIN/nodestack_plots/SVD/cm_maxamp overwritten
SVD
Sorting by distance
/Users/marinedenolle/BASIN/nodestack_plots/SVD/cm_maxamp overwritten
SVD
Sorting by distance
/Users/marinedenolle/BASIN/nodestack_plots/SVD/cm_maxamp overwritten
SVD
Sorting by distance
/Users/marinedenolle/BASIN/nodestack_plots/SVD/cm_maxamp overwritten
SVD
Sorting by distance
/Users/marinedenolle/BASIN/nodestack_plots/SVD/cm_maxamp overwritten
SVD
Sorting by distance
/Users/marinedenolle/BASIN/nodestack_plots/SVD/cm_max

HDF5-DIAG: Error detected in HDF5 (1.12.1) thread 1:
  #000: H5O.c line 128 in H5Oopen(): unable to open object
    major: Object header
    minor: Can't open object
  #001: H5VLcallback.c line 5386 in H5VL_object_open(): object open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 5353 in H5VL__object_open(): object open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_object.c line 58 in H5VL__native_object_open(): unable to open object by name
    major: Object header
    minor: Can't open object
  #004: H5Oint.c line 625 in H5O_open_name(): object not found
    major: Object header
    minor: Object not found
  #005: H5Gloc.c line 442 in H5G_loc_find(): can't find object
    major: Symbol table
    minor: Object not found
  #006: H5Gtraverse.c line 837 in H5G_traverse(): internal path traversal failed
    major: Symbol table
    minor: Object not found
  #007: H5Gtraverse.c line 613 in H5G__travers